# Introduction
        First of all, I choice this "Airline Passenger Satisfaction" dataset because of airline statistics intrigued me. Also I choice the "Flight Distance" column because I think that can find there is interesting statistics relations between that passenger satisfaction and flight distance. This column shows us the flight distance of all passengers. According to the column information mean is 119k, standard deviation is 997 and quantiles are Q1 = 414, Q2(Median) = 844, Q3 = 1744. Also, we see the minimum value is 31 and the maximum value is 4983.
   
<font color = 'purple'>
Content:
      
1. [ Mean of Column Data ](#1)
2. [ Median of Column Data ](#2)
3. [ Variance, Standard Deviation and Standard Error ](#3)  
4. [ Decide the Shape of Distribution ](#4)  
5. [ Find Outliers ](#5)  
6. [Graph the Column Data and Comment](#6)
7. [Boxplot](#7)

    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import statistics
import math

import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

import seaborn as sns

from collections import Counter

import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/data_dictionary.csv")
test_df = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/airline_passenger_satisfaction.csv")

<a id = "1"></a>
# Mean of Column Data

    To find a mean of column data, we have to sum all elements of the column data and divided by the column length.

In [ ]:
def mean(df):
    arr = np.array(df)
    mean = 0

    df_size = test_df.set_index(df)
    #print(len(df_size))
    n = len(df_size)

    for b in arr:
        mean = (mean + b)

    mean = mean / n

    return mean
    
print("Mean:", mean(test_df["Flight Distance"]))

<a id = "2"></a>
# Median of Column Data

        To find the median value of the column, we have to sort the dataset and check if the dataset length is even or odd. If the dataset length is an odd number, the median value will be the middle value of the dataset. If the dataset length is an even number, the median value will be the average of the two middle values.

In [ ]:
def median(df):
    sortedDf = sorted(df)
    dfLen = len(df)
    middle = (dfLen - 1) // 2
   
    if (dfLen % 2 == 0):
        return (sortedDf[middle] + sortedDf[middle + 1])/2.0
        
    else:
        return sortedDf[middle]
      

print("Median:", median(test_df["Flight Distance"]))

<a id = "3"></a>
# Variance, Standard Deviation and Standard Error

### Variance: 
    Variance tells you the degree of spread in your data set. The more spread the data, the larger the variance is in relation to the mean. A large variance means that the values have a large deviation from the arithmetic mean. To calculate variance, first of all we will subtract the mean from each value and square the results obtained. After that, we have to sum all squares, than finally divede the sum of squares by n(when you work with population).

### Standard Deviation:

    The standard deviation is the average amount of variability in your dataset. The standard deviation shows us on average, how far each value lies from the mean. If the value of the standard deviation ıs hıght that means values are generally far from the mean. It is calculated by squaring the variance.
    

### Standard Error:

    The standard error tells us, how different the population mean is likely to be from a sample mean. It is calculated by dividing the standard deviation by the square root of the number of elements.
    

In [ ]:
def variance(df):
    # Number of observations
    n = len(df)
    # Mean of the data
    mean = sum(df) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in df]
    # Variance
    variance = sum(deviations) / n  
    return variance

print("Variance:",variance(test_df["Flight Distance"]))

In [ ]:
def stdev(df):
    var = variance(df)
    std_dev = math.sqrt(var)
    return std_dev

print("Standart Deviation:", stdev(test_df["Flight Distance"]))

In [ ]:
def std_error(df):
    df_size = test_df.set_index(["Flight Distance"])
    #print(len(df_size))
    n = len(df_size)
    std_error = stdev(test_df["Flight Distance"]) / np.sqrt(n)
    return std_error

print("Standart Error:", std_error(test_df["Flight Distance"]))

<a id = "4"></a>
# Decide the Shape of Distribution

    Shape of distribution type is determined by the values of the average and the median. If the mean bigger than median, the shape of distribution will be Right-Skewed(Positively Skewed) distribution, if median bigger than mean, the shape of distribution will be Left-Skewed(Negatively Skewed) distribution. If mean and median are equal than the shape of distribution will be Normal(Symmetric) distribution. Our case we have a Right-Skewed(Positively Skewed) distribution for "Flight Distance" column, because the mean bigger than the median.

In [ ]:
def dist(df):
    
    me = mean(df)
    med = median(df)
    
    if (me) >(med):
        print("The shape of distribution of data is Right-Skewed(Positively Skewed) distribution. ")
    elif (me) < (med):
        print("The shape of distribution of data is Left-Skewed(Negatively Skewed) distribution. ")
    elif (me) == (med):
        print("The shape of distribution of data is Normal(Symmetric) distribution. ")
        
dist(test_df["Flight Distance"])

<a id = "5"></a>
# Find Outliers

    The outliers are as known as the extreme values of the dataset. This means extreme values are different from all other values on the dataset. If we want a consistent statistical result, we should clear them, because they can cause a huge effect on your statistic. To find outliers we have to calculate lower fence and upper fence, First of all, we will subtract q1 from q3 to find IQR. After that, to find the lower fence we have to will subtract (1.5 * IQR) from q1 and to find the upper fence we have to sum Q3 with (1.5 * IQR). After we find fences, values which bigger than the upper fence and smaller than the lower fence are called outliers.

In [ ]:
def detect_outlier(df, features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        q1 = np.percentile(df[c],25)
        # 3rd quartile
        q3 =  np.percentile(df[c],75)
        #IQR
        iqr = q3 - q1
        # Outlier step
        outlier_step = iqr * 1.5
        # Detect outliers and their indeces
        outlier_list_col = df[(df[c]< q1 - outlier_step) | (df[c]> q3 + outlier_step)].index
        # Store indeces
        outlier_indices.extend(outlier_list_col)
        # counter = finds how many times the element is repeated.
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items())
    
    return multiple_outliers

In [ ]:
test_df.loc[detect_outlier(test_df, ["Flight Distance"])]


<a id = "6"></a>
# Graph the Column Data and Make Comment

In [ ]:
df = px.data.tips()
fig = px.histogram(df, x=test_df["Flight Distance"])
fig.show()


According to histogram, we can say the histogram shape is Right-Skewed(Positively Skewed) distribution. This is because the mean bigger than median. Also we can see the number of flight distance which is most repetitive as a mod.

![](http://)<a id = "7"></a>
# Boxplot

In [ ]:
test_df.boxplot(column = "Flight Distance")
plt.show()

Fristly, when we look at the boxplot, we can see Q1, Q2 and Q3 value, median, maximum and minimum value, and also outliers. We see that all the outliers are above the maximum value. Also, according to the shape of the box plot, we can say that it has the shape of the distribution that we know as the Right Skewed (Positive Skewed) distribution.
